# Accessing Yelp API

Kristan Bryan Simbulan

## Initialization

In [1]:
# Install Yelp API
!pip install yelpapi

# Install tmdbsimple (only need to run once)
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [3]:
# Load API credentials
with open('/Users/kristansimbulan/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate Yelp API variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [7]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

## Accessing data from Yelp

In [5]:
# Intitiate 'location' and 'term' variables containing search keywords for location and term, respectively
Location = "New York"
Term = "Pasta"

In [6]:
## Specify JSON file for saving data
JSON_file = r"/Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json"
print(f'data will be saved to: {JSON_file}')

data will be saved to: /Users/kristansimbulan/Documents/GitHub/Working-with-Yelp-API/yelp-search-new-york-pasta.json


In [ ]:
## Create a new empty json file (delete the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

In [ ]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

In [6]:
# use the yelp_api variable's search_query method to perform API call
results_p1 = yelp_api.search_query(location = Location,
                                term = Term)

results_p1.keys()

dict_keys(['businesses', 'total', 'region'])

In [7]:
# use the yelp_api variable's search_query method to perform API call
results_p2 = yelp_api.search_query(location = Location,
                                term = Term,
                                offset = 20)

results_p2.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
# Convert business results into DataFrames
biz_p1 = pd.DataFrame(results_p1['businesses'])
biz_p2 = pd.DataFrame(results_p2['businesses'])

In [9]:
# Concatenate the two dataframes
businesses = pd.concat([biz_p1, biz_p2],
                      ignore_index=True)
display(businesses.head(3), businesses.tail(3))

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,5305,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,3186.380498
1,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3287,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.3,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
2,OCTiJwvjoK81WoDwsTkFvA,paesano-of-mulberry-street-new-york,Paesano of Mulberry Street,https://s3-media2.fl.yelpcdn.com/bphoto/yN9ARC...,False,https://www.yelp.com/biz/paesano-of-mulberry-s...,1496,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.1,"{'latitude': 40.718554, 'longitude': -73.997447}","[pickup, restaurant_reservation, delivery]",$$,"{'address1': '136 Mulberry St', 'address2': ''...",+12129651188,(212) 965-1188,1481.706653


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
37,h37t9rA06Sr4EetJjKrfzw,don-angie-new-york,Don Angie,https://s3-media2.fl.yelpcdn.com/bphoto/onJX6_...,False,https://www.yelp.com/biz/don-angie-new-york?ad...,845,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.73778, 'longitude': -74.00197}",[delivery],$$$,"{'address1': '103 Greenwich Ave', 'address2': ...",+12128898884,(212) 889-8884,3657.076882
38,QQpQujXQGbZBK6BeAokzDA,pomp-and-circumstance-brooklyn,Pomp and Circumstance,https://s3-media1.fl.yelpcdn.com/bphoto/SmGYpr...,False,https://www.yelp.com/biz/pomp-and-circumstance...,167,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.9,"{'latitude': 40.71427259634444, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '577 Lorimer St', 'address2': '',...",+17183874861,(718) 387-4861,3889.917598
39,DynFEEWtMX76qeqviUbOdA,mille-nonne-new-york-2,Mille Nonne,https://s3-media2.fl.yelpcdn.com/bphoto/Ykxj0h...,False,https://www.yelp.com/biz/mille-nonne-new-york-...,66,"[{'alias': 'italian', 'title': 'Italian'}]",4.7,"{'latitude': 40.717949, 'longitude': -73.988181}","[delivery, pickup]",NaN,"{'address1': '88 Essex St', 'address2': 'Ste 4...",+16466698957,(646) 669-8957,1491.810113


In [10]:
# Convert businesses dataframe to JSON
businesses_json = businesses.to_json(orient='records')

## Save the JSON data to the JSON file

In [13]:
# save the business data to the JSON file
with open(JSON_file, 'w') as file:
    json.dump(businesses_json, file)